# Fluidity Local: Stress Relaxation with Thixotropic Aging

**Learning Objectives:**
- Fit FluidityLocal to stress relaxation data from thixotropic yield-stress fluids
- Understand non-exponential relaxation driven by evolving fluidity f(t)
- Track accelerating relaxation from structural recovery (aging: f → f_eq)
- Compare FluidityLocal vs Maxwell-type exponential decay

**Prerequisites:** basic/01, bayesian/01, fluidity/01 (flow curve)

**Runtime:** Fast ~2 min (num_chains=1), Full ~5 min (num_chains=4)

## 1. Setup

In [ ]:
import sys

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install -q rheojax
    import os

    os.environ["JAX_ENABLE_X64"] = "true"
    print("RheoJAX installed successfully.")

In [ ]:
%matplotlib inline
import json
import os
import time
import warnings

import arviz as az
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

from rheojax.core.jax_config import safe_import_jax, verify_float64
from rheojax.models.fluidity import FluidityLocal

# Add utils to path
if "utils" not in sys.path:
    utils_path = os.path.join("..", "utils")
    sys.path.insert(0, utils_path)

from fluidity_tutorial_utils import (
    generate_synthetic_relaxation,
    get_output_dir,
    print_convergence_summary,
    print_parameter_comparison,
    save_fluidity_results,
)

jax, jnp = safe_import_jax()
verify_float64()

warnings.filterwarnings("ignore", category=FutureWarning)
print(f"JAX version: {jax.__version__}")
print(f"Devices: {jax.devices()}")

## 2. Theory: Fluidity Stress Relaxation

After a step strain γ₀ is imposed at t=0, the stress relaxes as:

$$\frac{d\sigma}{dt} = -G\sigma f(t), \quad \frac{df}{dt} = \frac{f_{eq} - f}{\theta}$$

where:
- **G**: Elastic modulus (Pa)
- **f(t)**: Time-dependent fluidity (1/(Pa·s))
- **f_eq**: Equilibrium fluidity at rest (1/(Pa·s))
- **θ**: Aging time scale (s)

**Key Behavior:**
1. **Initial fast decay**: f starts high (from prior flow), σ decays rapidly
2. **Aging slowdown**: f decreases toward f_eq as structure recovers
3. **Accelerating relaxation**: Lower f means slower stress relaxation → stretched exponential signature

**Contrast with Maxwell:** Standard viscoelastic models predict σ(t) = σ₀ exp(-t/τ) with constant relaxation time τ. Fluidity models exhibit **time-dependent relaxation** due to evolving microstructure.

**Thixotropic Signature:** The relaxation modulus G(t) = σ(t)/γ₀ decays faster at short times (high f, recent flow) and slower at long times (low f, structural recovery).

## 3. Load Data

We'll use synthetic data generated from a calibrated FluidityLocal model. In practice, you would load experimental relaxation data from laponite clays or other thixotropic materials.

In [ ]:
# Generate synthetic relaxation data
# (In real workflow, load from clay data: load_clay_relaxation(aging_time=60))

# Create a reference model with thixotropic parameters
model_ref = FluidityLocal()
model_ref.parameters.set_value("G", 1000.0)  # Pa
model_ref.parameters.set_value("tau_y", 50.0)  # Pa
model_ref.parameters.set_value("K", 10.0)  # Pa·s^n
model_ref.parameters.set_value("n_flow", 0.5)  # dimensionless
model_ref.parameters.set_value("f_eq", 0.001)  # 1/(Pa·s)
model_ref.parameters.set_value("f_inf", 0.01)  # 1/(Pa·s)
model_ref.parameters.set_value("theta", 10.0)  # s
model_ref.parameters.set_value("a", 0.1)  # dimensionless
model_ref.parameters.set_value("n_rejuv", 1.0)  # dimensionless
model_ref.fitted_ = True

# Generate synthetic data
time, stress = generate_synthetic_relaxation(
    model_ref, sigma_0=500.0, t_end=100.0, n_points=80, noise_level=0.05, seed=42
)

print(f"Data: {len(time)} points, t=[{time.min():.3f}, {time.max():.1f}] s")
print(f"Stress range: [{stress.min():.1f}, {stress.max():.1f}] Pa")

In [ ]:
# Visualize relaxation data
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Log-log plot
ax1.loglog(time, stress, "ko", markersize=5, label="Data")
ax1.set_xlabel("Time [s]")
ax1.set_ylabel("Stress [Pa]")
ax1.set_title("Stress Relaxation (Log-Log)")
ax1.legend()
ax1.grid(True, alpha=0.3, which="both")

# Semi-log plot (to show deviation from exponential)
ax2.semilogy(time, stress, "ko", markersize=5, label="Data")
ax2.set_xlabel("Time [s]")
ax2.set_ylabel("Stress [Pa]")
ax2.set_title("Stress Relaxation (Semi-Log)")
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
display(fig)
plt.close(fig)

## 4. NLSQ Fitting

### 4.1 Fit FluidityLocal to Relaxation Data

In [ ]:
model = FluidityLocal()

t0_fit = time.time()
model.fit(time, stress, test_mode="relaxation", sigma_0=stress[0], method='scipy')

# Compute fit quality
stress_pred = model.predict(t, test_mode="relaxation", gamma_0=gamma_0)
metrics = compute_fit_quality(stress_data, stress_pred)
t_nlsq = time.time() - t0_fit

print(f"NLSQ fit time: {t_nlsq:.2f} s")
print(f"R²: {metrics['R2']:.6f}")
print(f"RMSE: {metrics['RMSE']:.4g} Pa")
print("\nFitted parameters:")
param_names = ["G", "tau_y", "K", "n_flow", "f_eq", "f_inf", "theta", "a", "n_rejuv"]
for name in param_names:
    val = model.parameters.get_value(name)
    print(f"  {name:10s} = {val:.4g}")

In [ ]:
# Predict and compare
t_fine = np.logspace(np.log10(time.min()) - 0.3, np.log10(time.max()) + 0.3, 200)
stress_pred = model.predict(t_fine)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Log-log
ax1.loglog(time, stress, "ko", markersize=6, label="Data")
ax1.loglog(t_fine, stress_pred, "-", lw=2, color="C0", label="FluidityLocal fit")
ax1.set_xlabel("Time [s]")
ax1.set_ylabel("Stress [Pa]")
ax1.set_title(f"Fluidity Relaxation Fit — R²={metrics['R2']:.4f}")
ax1.legend()
ax1.grid(True, alpha=0.3, which="both")

# Semi-log (curvature = non-exponential)
ax2.semilogy(time, stress, "ko", markersize=6, label="Data")
ax2.semilogy(t_fine, stress_pred, "-", lw=2, color="C0", label="FluidityLocal fit")

# Reference exponential for comparison (Maxwell with tau = 1/(G*f_eq))
G_fit = model.parameters.get_value("G")
f_eq_fit = model.parameters.get_value("f_eq")
tau_maxwell = 1.0 / (G_fit * f_eq_fit + 1e-30)
stress_maxwell = stress[0] * np.exp(-t_fine / tau_maxwell)
ax2.semilogy(
    t_fine, stress_maxwell, "--", lw=1.5, color="gray", alpha=0.6, label="Maxwell (exponential)"
)

ax2.set_xlabel("Time [s]")
ax2.set_ylabel("Stress [Pa]")
ax2.set_title("Deviation from Exponential (Thixotropic Signature)")
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
display(fig)
plt.close(fig)

### 4.2 Fluidity Evolution During Relaxation

To visualize the thixotropic aging mechanism, we simulate the fluidity evolution f(t) during relaxation.

In [ ]:
# Simulate fluidity evolution using diffrax
from rheojax.models.fluidity._kernels import fluidity_local_ode_rhs
import diffrax

# Get fitted parameters
p = model.get_parameter_dict()
f_init = p["f_inf"]  # Start with high fluidity (just flowed)
sigma_init = stress[0]

# ODE arguments
args = {
    "G": p["G"],
    "f_eq": p["f_eq"],
    "f_inf": p["f_inf"],
    "theta": p["theta"],
    "a": p["a"],
    "n_rejuv": p["n_rejuv"],
    "gamma_dot": 0.0,  # No flow during relaxation
}

# Initial state: [sigma, f]
y0 = jnp.array([sigma_init, f_init])

# Time span
t_span = jnp.array([time.min(), time.max()])
saveat = diffrax.SaveAt(ts=jnp.asarray(t_fine))

# Solve ODE
term = diffrax.ODETerm(fluidity_local_ode_rhs)
solver = diffrax.Tsit5()
stepsize_controller = diffrax.PIDController(rtol=1e-6, atol=1e-8)

sol = diffrax.diffeqsolve(
    term,
    solver,
    t0=t_span[0],
    t1=t_span[1],
    dt0=0.01,
    y0=y0,
    args=args,
    saveat=saveat,
    stepsize_controller=stepsize_controller,
    max_steps=1_000_000,
)

stress_sim = np.array(sol.ys[:, 0])
fluidity_sim = np.array(sol.ys[:, 1])

print(f"Fluidity range: [{fluidity_sim.min():.4g}, {fluidity_sim.max():.4g}] 1/(Pa·s)")
print(f"f_eq (target): {p['f_eq']:.4g} 1/(Pa·s)")

In [ ]:
# Plot stress and fluidity evolution
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8), sharex=True)

# Stress
ax1.loglog(time, stress, "ko", markersize=5, label="Data")
ax1.loglog(t_fine, stress_sim, "-", lw=2, color="C0", label="FluidityLocal")
ax1.set_ylabel("Stress [Pa]")
ax1.set_title("Stress Relaxation and Fluidity Evolution")
ax1.legend()
ax1.grid(True, alpha=0.3, which="both")

# Fluidity
ax2.semilogx(t_fine, fluidity_sim, "-", lw=2, color="C1", label="f(t)")
ax2.axhline(
    p["f_eq"], color="red", linestyle="--", alpha=0.5, lw=1.5, label=f"f_eq={p['f_eq']:.4g}"
)
ax2.axhline(
    p["f_inf"], color="blue", linestyle="--", alpha=0.5, lw=1.5, label=f"f_inf={p['f_inf']:.4g}"
)
ax2.set_xlabel("Time [s]")
ax2.set_ylabel("Fluidity [1/(Pa·s)]")
ax2.set_title("Aging: f → f_eq (Structural Recovery)")
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
display(fig)
plt.close(fig)

**Interpretation:**
- **Upper panel:** Stress decays non-exponentially (curved on semi-log plot)
- **Lower panel:** Fluidity f(t) decreases from f_inf (high, recent flow) toward f_eq (low, structural recovery)
- **Mechanism:** Lower f → slower stress relaxation → accelerating decay signature

## 5. Bayesian Inference with NUTS

### 5.1 Run NUTS

In [ ]:
initial_values = {name: model.parameters.get_value(name) for name in param_names}
print("Warm-start values:")
for name, val in initial_values.items():
    print(f"  {name:10s} = {val:.4g}")

NUM_WARMUP = 200
NUM_SAMPLES = 500
NUM_CHAINS = 1
# NUM_WARMUP = 1000; NUM_SAMPLES = 2000; NUM_CHAINS = 4  # production

t0 = time.time()
result = model.fit_bayesian(
    time,
    stress,
    test_mode="relaxation",
    sigma_0=stress[0],
    num_warmup=NUM_WARMUP,
    num_samples=NUM_SAMPLES,
    num_chains=NUM_CHAINS,
    initial_values=initial_values,
    seed=42,
)
t_bayes = time.time() - t0
print(f"\nBayesian inference time: {t_bayes:.1f} s")

### 5.2 Convergence Diagnostics

In [ ]:
converged = print_convergence_summary(result, param_names=param_names)

### 5.3 ArviZ Diagnostic Plots

In [ ]:
idata = result.to_inference_data()

# Focus on key fluidity parameters for clarity
key_params = ["G", "f_eq", "f_inf", "theta", "tau_y"]

axes = az.plot_trace(idata, var_names=key_params, figsize=(12, 8))
fig = axes.ravel()[0].figure
fig.suptitle("Trace Plots (Key Parameters)", fontsize=14, y=1.02)
plt.tight_layout()
display(fig)
plt.close(fig)

In [ ]:
axes = az.plot_pair(
    idata, var_names=["G", "f_eq", "theta"], kind="scatter", divergences=True, figsize=(9, 9)
)
fig = axes.ravel()[0].figure
fig.suptitle("Parameter Correlations (G, f_eq, θ)", fontsize=14, y=1.02)
plt.tight_layout()
display(fig)
plt.close(fig)

In [ ]:
axes = az.plot_forest(
    idata, var_names=key_params, combined=True, hdi_prob=0.95, figsize=(10, 5)
)
fig = axes.ravel()[0].figure
plt.tight_layout()
display(fig)
plt.close(fig)

### 5.4 Posterior Predictive Check

In [ ]:
posterior = result.posterior_samples
n_draws = min(200, len(posterior["G"]))

# Predict with posterior draws
pred_samples = []
for i in range(n_draws):
    # Build parameter array matching model_function signature
    params_i = jnp.array(
        [
            posterior["G"][i],
            posterior["tau_y"][i],
            posterior["K"][i],
            posterior["n_flow"][i],
            posterior["f_eq"][i],
            posterior["f_inf"][i],
            posterior["theta"][i],
            posterior["a"][i],
            posterior["n_rejuv"][i],
        ]
    )
    # Note: model_function may need sigma_0 in kwargs
    pred_i = model.model_function(
        jnp.asarray(t_fine), params_i, test_mode="relaxation", sigma_0=stress[0]
    )
    pred_samples.append(np.array(pred_i))

pred_samples = np.array(pred_samples)
pred_median = np.median(pred_samples, axis=0)
pred_lo = np.percentile(pred_samples, 2.5, axis=0)
pred_hi = np.percentile(pred_samples, 97.5, axis=0)

fig, ax = plt.subplots(figsize=(9, 6))
ax.fill_between(t_fine, pred_lo, pred_hi, alpha=0.3, color="C0", label="95% CI")
ax.loglog(t_fine, pred_median, "-", lw=2, color="C0", label="Posterior median")
ax.loglog(time, stress, "ko", markersize=5, label="Data")
ax.set_xlabel("Time [s]")
ax.set_ylabel("Stress [Pa]")
ax.set_title("Posterior Predictive Check")
ax.legend()
ax.grid(True, alpha=0.3, which="both")
plt.tight_layout()
display(fig)
plt.close(fig)

### 5.5 Parameter Comparison

In [ ]:
print_parameter_comparison(model, posterior, param_names=param_names)

## 6. Save Results

In [ ]:
save_fluidity_results(
    model, result, model_variant="local", protocol="relaxation", param_names=param_names
)

# Also save fluidity evolution for future reference
output_dir = get_output_dir("local", "relaxation")
fluidity_data = {"time": t_fine.tolist(), "fluidity": fluidity_sim.tolist()}
with open(os.path.join(output_dir, "fluidity_evolution.json"), "w") as f:
    json.dump(fluidity_data, f)
print(f"Fluidity evolution saved to {output_dir}/fluidity_evolution.json")

## Key Takeaways

1. **Non-exponential decay**: FluidityLocal predicts σ(t) decay faster than simple exponential due to time-varying fluidity f(t), reflecting thixotropic aging.

2. **Aging mechanism**: After flow stops (γ̇=0), fluidity f(t) decreases from f_inf (high, recent flow) toward f_eq (low, structural recovery). This is the structural aging signature.

3. **Accelerating relaxation**: Lower f → slower stress relaxation rate dσ/dt = -Gσf. The relaxation time τ_eff(t) = 1/(Gf(t)) increases with time, creating stretched exponential behavior.

4. **Contrast with Maxwell**: Standard viscoelastic models show straight lines on semi-log plots (exponential). Fluidity models show curvature due to evolving microstructure.

5. **Parameter identifiability**: Relaxation data primarily constrains G, f_eq, and θ (aging time). Flow parameters (tau_y, K, n) have weak influence but are needed for full model consistency.

**Next Steps:** Explore startup shear (NB 02) for stress overshoot from thixotropic breakdown, or creep (NB 03) for delayed yielding. Both protocols reveal complementary aspects of fluidity dynamics.